In [63]:
import pandas as pd
train_df = pd.read_csv("./dataset/train.csv",sep=";",names=['sentence','emotion'])
val_df = pd.read_csv("./dataset/val.csv",sep=";",names=['sentence','emotion'])
test_df = pd.read_csv("./dataset/test.csv",sep=";",names=['sentence','emotion'])
train_df.head(5),val_df.head(5),test_df.head(5)

(                                            sentence  emotion
 0                            i didnt feel humiliated  sadness
 1  i can go from feeling so hopeless to so damned...  sadness
 2   im grabbing a minute to post i feel greedy wrong    anger
 3  i am ever feeling nostalgic about the fireplac...     love
 4                               i am feeling grouchy    anger,
                                             sentence  emotion
 0  im feeling quite sad and sorry for myself but ...  sadness
 1  i feel like i am still looking at a blank canv...  sadness
 2                     i feel like a faithful servant     love
 3                  i am just feeling cranky and blue    anger
 4  i can have for a treat or if i am feeling festive      joy,
                                             sentence  emotion
 0  im feeling rather rotten so im not very ambiti...  sadness
 1          im updating my blog because i feel shitty  sadness
 2  i never make her separate from me because i do...

In [64]:
train_df.groupby(['emotion']).describe(),val_df.groupby(['emotion']).describe(),test_df.groupby(['emotion']).describe()

(         sentence                                                            \
             count unique                                                top   
 emotion                                                                       
 anger        2159   2159  i was feeling a little grumpy thinking about e...   
 fear         1937   1937  i do and it is really starting to make me feel...   
 joy          5362   5361  i feel more adventurous willing to take risks ...   
 love         1304   1304  i feel very nostalgic because i have enjoyed t...   
 sadness      4666   4666             i feel idiotic and wierd in this class   
 surprise      572    572  i feel you might be quite amazed if ahead of y...   
 
                
          freq  
 emotion        
 anger       1  
 fear        1  
 joy         2  
 love        1  
 sadness     1  
 surprise    1  ,
          sentence                                                            \
             count unique                   

In [10]:
emotion = train_df['emotion'].unique()
emotion

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [68]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

train_df['emotion_code'] = LE.fit_transform(train_df['emotion'])
val_df['emotion_code'] = LE.fit_transform(val_df['emotion'])
test_df['emotion_code'] = LE.fit_transform(test_df['emotion'])

train_df,val_df,test_df

(                                                sentence  emotion  \
 0                                i didnt feel humiliated  sadness   
 1      i can go from feeling so hopeless to so damned...  sadness   
 2       im grabbing a minute to post i feel greedy wrong    anger   
 3      i am ever feeling nostalgic about the fireplac...     love   
 4                                   i am feeling grouchy    anger   
 ...                                                  ...      ...   
 15995  i just had a very brief time in the beanbag an...  sadness   
 15996  i am now turning and i feel pathetic that i am...  sadness   
 15997                     i feel strong and good overall      joy   
 15998  i feel like this was such a rude comment and i...    anger   
 15999  i know a lot but i feel so stupid because i ca...  sadness   
 
        emotion_code  
 0                 4  
 1                 4  
 2                 0  
 3                 3  
 4                 0  
 ...             ...

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
stopwords = nltk.corpus.stopwords.words('english')

cleaned_text = []

for data in train_df['sentence']:
    data = list(set(data.split(" ")) - set(stopwords))
    cleaned_text.append(" ".join(data))


TFIDF = TfidfVectorizer(analyzer='word',stop_words='english',ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)
model = TFIDF.fit_transform(cleaned_text)

word = TFIDF.get_feature_names()
word 

sentence_tfidf_df = pd.DataFrame(data=model.toarray(),columns=TFIDF.get_feature_names())

In [70]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(sentence_tfidf_df,train_df['emotion_code'])

KNeighborsClassifier()

In [103]:
import helper

vectorized = TFIDF.transform(val_df['sentence'])
vectorized

<2000x14870 sparse matrix of type '<class 'numpy.float64'>'
	with 14727 stored elements in Compressed Sparse Row format>

In [104]:
# Validation
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(knn.predict(vectorized),val_df['emotion_code'])

accuracy * 100

14.05